# Training an artificial neural network

Now, let's dig a little bit deeper into some of the concepts, such as the logistic cost function and the backpropagation algorithm.

> If you are unfamiliar with calculus or need a brief refresher, consider reading this text as an additional supporting resource before continuing with this course. [Here](https://sebastianraschka.com/pdf/books/dlb/appendix_d_calculus.pdf)

## Computing the logistic cost function

The logistic cost function that will be implemented as the `_compute_cost` method is actually pretty simple to follow since it is the same cost function that we described in the *logistic regression*

\begin{equation*}
J \left( w \right) = -\sum^n_{i=1} y^{[i]} log \left(a^{[i]} \right) + \left(1 - y^{[i]} \right) log \left(1 - a^{[i]} \right)
\end{equation*}

Here, $a^{[i]}$ is the sigmoid activation of the ith sample in the dataset, which we compute in the forward propagation step:

\begin{equation*}
a^{[i]} = \phi \left( z^{[i]} \right)
\end{equation*}

Again, note that in this context, the superscript [i] is an index for training samples, not layers.

Now, let's add a regularization term, which allows us to reduce the degree of overfitting. L2 regularization term is defined as follows:

\begin{equation*}
L2 = \lambda \| w \|^2_2 = \lambda \sum^m_{j=1} w^2_j
\end{equation*}

By adding the L2 regularization term to our logistic cost function, we obtain the following equation:

\begin{equation*}
J \left( w \right) = - \left[ \sum^n_{i=1} y^{[i]} log \left(a^{[i]} \right) + \left(1 - y^{[i]} \right) log \left(1 - a^{[i]} \right) \right] + \frac{\lambda}{2} \| w \|^2_2
\end{equation*}

Since we will  implement an MLP for multiclass classification that returns an output vector of t elements that we need to compare to the $t \times 1$ dimensional target vector in the one-hot encoding representation, for example, the activation of the third layer and the target class (here, class 2) for a particular sample may look like this:

\begin{equation*}
a^{(out)} = \begin{bmatrix}
0.1 \\
0.9 \\
\vdots \\
0.3
\end{bmatrix}, y = \begin{bmatrix}
0 \\
1 \\
\vdots \\
0
\end{bmatrix}
\end{equation*}

Thus, we need to generalize the logistic cost function to all t activation units in our network. Thus, the cost function (without the regularization term) becomes the following:

\begin{equation*}
 \left( w \right) = - \sum^n_{i=1} \sum^t_{j=1} y^{[i]}_j log \left(a^{[i]}_j \right) + \left(1 - y^{[i]}_j \right) log \left(1 - a^{[i]}_j \right)
\end{equation*}

Here, again, the superscript **(i)** is the index of a particular sample in our training set.

The following generalized regularization term may look a little bit complicated at first, but here we are just calculating the sum of all weights of an $l$ layer (without the bias term) that we added to the first column:

\begin{equation*}
J \left( w \right) = - \left[ \sum^n_{i=1} \sum^t_{j=1} y^{[i]}_j log \left(a^{[i]}_j \right) + \left(1 - y^{[i]}_j \right) log \left(1 - a^{[i]}_j \right) \right] + \frac{\lambda}{2} \sum^{L-1}_{l=1} \sum^{u_l}_{i=1} \sum^{u_{l+1}}_{j=1} \left( w^{(l)}_{j, i} \right)^2
\end{equation*}

Here, $u_1$ refers to the number of units in a given layer $l$, and the following expression represents the penalty term:

\begin{equation*}
\frac{\lambda}{2} \sum^{L-1}_{l=1} \sum^{u_l}_{i=1} \sum^{u_{l+1}}_{j=1} \left( w^{(l)}_{j, i} \right)^2
\end{equation*}

Remember that our goal is to minimize the cost function $J \left(W \right)$; thus we need to calculate the partial derivative of the parameters $W$ with respect to each weight for every layer in the network:

\begin{equation*}
\frac{\partial}{\partial w^{(l)}_{j, i}} J \left( W \right)
\end{equation*}

Note that $W$ consists of multiple matrices. In a multilayer perceptron with one hidden unit, we have the weight matrix $W^{(h)}$, which connects the input to the hidden layer, and $W^{(out)}$, which connects the hidden layer to the output layer. An intuitive visualization of the three-dimensional tensor $W$ is provided in the following figure:

<img src="img/fig4.jpg" width="400" height="300" >


In this simplified figure, it may seem that both $W^{(h)}$ and $W^{(out)}$ have the same number of rows and columns, which is typically not the case unless we initialize an MLP with the same number of hidden units, output units, and input features.

## backpropagation

Although backpropagation was rediscovered and popularized more than 30 years ago, it still remains one of the most widely used algorithms to train artificial neural networks very efficiently. 

  > Readings
  > * Learning representations by back-propagating errors, D. E. Rumelhart, G. E. Hinton, and R. J. Williams, Nature, 323: 6088, pages 533–536, 1986
  > * [Who Invented Backpropagation?](http://people.idsia.ch/~juergen/who-invented-backpropagation.html)
  
In essence, we can think of backpropagation as a very computationally efficient approach to compute the partial derivatives of a complex cost function in multilayer neural networks. Here, our goal is to use those derivatives to learn the weight coefficients for parameterizing such a multilayer artificial neural network. The challenge in the parameterization of neural networks is that we are typically dealing with a very large number of weight coefficients in a high-dimensional feature space. In contrast to cost functions of single-layer neural networks such as Adaline or logistic regression. the error surface of a neural network cost function is not convex or smooth with respect to the parameters. There are many bumps in this high-dimensional cost surface (local minima) that we have to overcome in order to find the global minimum of the cost function.

In a previous section, we saw how to calculate the cost as the difference between the activation of the last layer and the target class label. Now, we will see how the backpropagation algorithm works to update the weights in our MLP model from a mathematical perspective, which it must be  implemented in the `Backpropagation` section inside the `fit` method.

First need to apply forward propagation in order to obtain the activation of the output layer, which we formulated as follows:

\begin{equation*}
\begin{matrix}
Z^{(h)} = A^{(in)}W^{(h)} & \text{net input of the hidden layer} \\
A^{(h)} = \phi \left( Z^{(h)} \right) & \text{activation of the hidden layer} \\
Z^{(out)} = A^{(h)}W^{(out)} & \text{net input of the output layer} \\
A^{(out)} = \phi \left(Z^{(out)} \right) & \text{activation of the output layer}
\end{matrix}
\end{equation*}

Concisely, we just forward-propagate the input features through the connection in the network, as shown in the following illustration:

<img src="img/fig5.jpg" width="400" height="300" >

In backpropagation, we propagate the error from right to left. We start by calculating the error vector of the output layer:

\begin{equation*}
\delta^{(out)} = a^{(out)} - y
\end{equation*}

Here, $y$ is the vector of the true class labels.

Next, we calculate the error term of the hidden layer:

\begin{equation*}
\delta^{(h)} = \delta^{(out)} \left( W^{(out)} \right)^T \odot \frac{\partial \phi \left( z^{(h)} \right)}{\partial z^{(h)}}
\end{equation*}

Here, $\frac{\partial \phi \left( z^{(h)} \right)}{\partial z^{(h)}}$ is simply the derivative of the sigmoid activation function.

\begin{equation*}
\frac{\partial \phi \left(z \right)}{\partial z} = \left( a^{(h)} \odot \left( 1 -a^{(h)} \right) \right)
\end{equation*}

Note that the $\odot$  symbol means element-wise multiplication in this context.

Next, we compute the $\delta^{(h)}$ layer error matrix (`sigma_h`) as follows:

\begin{equation*}
\delta^{(h)} = \delta^{(out)} \left( W^{(out)} \right)^T \odot \left( a^{(h)} \odot \left( 1 - a^{(h)} \right) \right)
\end{equation*}

To better understand how we computed this $\delta^{(h)}$ term, let's walk through it in more detail. In the preceding equation, we used the transpose $\left( W^{(out)} \right)^T$ of the $h \times t$-dimensional matrix $W^{(out)}$. Here, $t$ is the number of output class labels and $h$ is the number of hidden units. The matrix multiplication between the $n \times t$-dimensional $\delta^{(out)}$ matrix and the $t \times h$-dimensional matrix $ \left( W^{(out)} \right)^T$, results in an $n \times t$-dimensional matrix that we multiplied elementwise by the sigmoid derivative of the same dimension to obtain the $n \times t$-dimensional matrix $\delta^{(h)}$.

Eventually, after obtaining the $\delta$ terms, we can now write the derivation of the cost function as follows:

\begin{equation*}
\begin{matrix}
\frac{\partial}{\partial w^{(out)}_{i, j}} J \left( W \right) = a^{(h)}_j \delta^{(out)}_i \\
\frac{\partial}{\partial w^{(h)}_{i, j}} J \left( W \right) = a^{(in)}_j \delta^{(h)}_i
\end{matrix}
\end{equation*}

Next, we need to accumulate the partial derivative of every node in each layer and the error of the node in the next layer. However, remember that we need to compute $\Delta^{(l)}_{i, j}$ for every sample in the training set. Thus, it is easier to implement it as a vectorized version:

\begin{equation*}
\begin{matrix}
\Delta^{(h)} = \Delta^{(h)} + \left( A^{(in)} \right)^T \delta^{(h)} \\
\Delta^{(out)} = \Delta^{(out)} + \left( A^{(h)} \right)^T \delta^{(out)}
\end{matrix}
\end{equation*}

And after we have accumulated the partial derivatives, we can add the regularization term:

\begin{equation*}
\Delta^{(l)} := \Delta^{(l)} + \lambda^{(l)} \text{( except for the bias term)}
\end{equation*}

Lastly, after we have computed the gradients, we can now update the weights by taking an opposite step towards the gradient for each layer $l$:

\begin{equation*}
W^{(l)} := W^{(l)} - \eta \Delta^{(l)}
\end{equation*}

To bring everything together, let's summarize backpropagation in the following figure:

<img src="img/fig6.jpg" width="400" height="300" >